In [1]:
import pandas as pd
import re
import calendar
from IPython.display import clear_output
import sqlite3 as sq
import sqlalchemy
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

TN_stations = ['clarksville', 'lockland', 'cookeville', 'shelby-farms', 'kingsport',
               'hville', 'air-lab', 'loretto', 'columbia', 'maryvill', 'look-rock-gsmnp',
               'harriman', 'riverside', 'loudonms', 'jackson', 'athens', 'dyerteom']

columns = ['City', 'Station', 'Year', 'Month', 'Day', 'Time', 'AQI', 'Pollution_Range',
           'Temp_f', 'Pressure', 'Humidity', 'Wind_MPH']

# EMPTY LIST FOR APPENDING WEATHER DATA:
report = []

# SET INITIAL VALUES FOR OUTPUT COUNTER:
record_count = 0
total_stations = len(TN_stations)

# LOOP THROUGH THE HTML TABLES TO BUILD DATAFRAME & OUTPUT:
for i, city in enumerate(TN_stations):
    try:
        if (i >= total_stations):
            record_count = 1

        # ",end=''" ALLLOWS THE NEXT LINE TO CONTINUE ON THE SAME LINE:
        print(f"Displaying City {record_count} of {total_stations} | ", end='')
        # '\033[91' ADDS RED FONT, '\033[0m' ENDS THE SETTING:
        print('\033[91m'+f"Next Station: {city}"+'\033[0m')

        record_count += 1

        df = pd.read_html(f'https://aqicn.org/city/usa/tennessee/{city}')

        # INITIAL CITY STRING:
        location = df[2][0][0][:-1]
        # SEPARATE THE CITY NAME FROM THE STRING:
        location = location[location.index(':'):][2:].split(',')[0]

        # ACCESS THE LEGEND ON THE WEBSITE THAT EXPLAINS THE POLLUTION RATINGS:
        legend = df[17].set_index('AQI').rename_axis(None)

        # TIMESTAMP FOR THE MOST RECENT STATION UPDATE:
        datestring = df[3][1][0]

        # ADDRESS ERRORS WHEN THE STATIONS DO NOT REPORT:
        try:
            date = re.findall(',([^"]*),', datestring)[0][5:][:-1]
        except:
            date = 'No report date on record'

        # LOCATE THE TABLE THAT HOUSES ALL THE WEATHER DATA:
        info_table = df[4][[0, 1]]

        # RETRIEVE THE TABLE DATA FOR EACH CITY:
        # *****************************************************************
        try:
            aqi = int(info_table.loc[info_table[0] == 'PM2.5 AQI', 1])
        except:
            aqi = 'No AQI reported'
        # *****************************************************************
        try:
            celcius = int(info_table.loc[info_table[0] == 'Temp.', 1])
            fahrenheit = round((9/5*(celcius)+32), 2)
        except:
            fahrenheit = 'No temperature reported'
        # *****************************************************************
        try:
            pressure = int(info_table.loc[info_table[0] == 'Pressure', 1])
        except:
            pressure = 'No pressure reported'
        # *****************************************************************
        try:
            humidity = int(info_table.loc[info_table[0] == 'Humidity', 1])
        except:
            humidity = 'No humidity reported'
        # *****************************************************************
        try:
            wind = int(info_table.loc[info_table[0] == 'Wind', 1])
        except:
            wind = 'No wind reported'
        # *****************************************************************

        # PRINT OUT THE AQI CATAGORY RANGES:
        try:
            if aqi in range(0, 51):
                pollution = legend.loc[legend.index == '0 - 50',
                                       'Air Pollution Level'].to_string()
            elif aqi in range(51, 101):
                pollution = legend.loc[legend.index == '51 -100',
                                       'Air Pollution Level'].to_string()
            elif aqi in range(101, 151):
                pollution = legend.loc[legend.index == '101-150',
                                       'Air Pollution Level'].to_string()
            elif aqi in range(151, 201):
                pollution = legend.loc[legend.index == '151-200',
                                       'Air Pollution Level'].to_string()
            elif aqi in range(201-301):
                pollution = legend.loc[legend.index == '201-300',
                                       'Air Pollution Level'].to_string()
            else:
                pollution = legend.loc[legend.index == '300+',
                                       'Air Pollution Level'].to_string()
        except:
            pollution = 'No pollution level to report'

        # CORRECT STATION NAMES TO CITY NAMES:
        if city == 'lockland':
            location = 'Nashville'
        elif city == 'shelby-farms':
            location = 'Memphis'
        elif city == 'hville':
            location = 'Hendersonville'
        elif city == 'air-lab':
            location = 'Knoxville'
        elif city == 'maryvill':
            location = 'Maryville'
        elif city == 'look-rock-gsmnp':
            location = 'Look Rock/GSMNP'
        elif city == 'riverside':
            location = 'Chattanooga'
        elif city == 'loudonms':
            location = 'Loudon'
        elif city == 'dyerteom':
            location = 'Dyersburg'
        else:
            location = location.capitalize()

        # PARSE DATES:

        month = date.split(',')[1].strip().split(' ')[0]
        month = list(calendar.month_abbr).index(month)
        day = int(date.split(',')[1].strip().split(' ')[1][:-2])
        year = int(date.split(',')[1].strip().split(' ')[2])
        time = date.split(',')[2].strip()

        # PUSH & APPEND THE DATA TO THE EMPTY DATAFRAME:
        report.append([location, city, year, month, day, time, aqi, pollution, fahrenheit,
                       pressure, humidity, wind])

        # SHOW EACH CITY ONE AT A TIME, PAUSING ON EACH OUTPUT:
        clear_output(wait=True)
        import time
        time.sleep(2)

        print('\033[1m'+f'{location}, TN'+'\033[0m')
        print('\033[91m'+f'Station Name: {city}'+'\033[0m')
        print(f'{date}')
        print('')
        print('\033[94m'+'Real-time Air Quality Index (AQI): ' +
              '\033[0m', end='')
        print(f'{aqi}')

        # ADD A COLOR-CODED CIRCLE ACCORDING TO EACH WARNING LEVEL:
        if aqi < 51:
            print("\U0001f7e2", end=' ')
        elif aqi < 101:
            print("\U0001f7e1", end=' ')
        elif aqi < 151:
            print("\U0001f7e0", end=' ')
        elif aqi < 201:
            print("\U0001f534", end=' ')
        elif aqi < 301:
            print("\U0001f7e4", end=' ')
        else:
            print("\U0001f635", end=' ')

        print(f'{pollution}\n')

        # '\033[92m' = CYAN
        print('\033[92m'+'Current Temperature: '+'\033[0m', end='')
        print(f'{fahrenheit}° Fahrenheit')

        print('\033[92m'+'Atmospheric Pressure: '+'\033[0m', end='')
        print(f'{pressure}')

        print('\033[92m'+'Humidity: '+'\033[0m', end='')
        print(f'{humidity}')

        print('\033[92m'+'Wind in mph: '+'\033[0m', end='')
        print(f'{wind}\n')
    except:
        clear_output(wait=True)
        time.sleep(2)
        print('')
        message = df[3][1][0].split('2021try')[0].replace(
            'on', 'since').replace('U', ' u').strip()
        print('\033[1m'+f'{location}, TN'+'\033[0m')
        print('\033[91m'+f'Station Name: {city}'+'\033[0m')
        print(message)
        print('SKIPPING...')
        print('')
        pass

# BUILD THE NEW DATAFRAME:
weatherAQI = pd.DataFrame(report, columns=columns)

# COMBINE THE DATE COLUMNS & REMOVE AM OR PM FROM THE TIME:
combined_date = pd.to_datetime(weatherAQI[['Year', 'Month', 'Day']])

weatherAQI.insert(2, 'Date', combined_date)
weatherAQI = weatherAQI.drop(['Year', 'Month', 'Day'], axis=1)
weatherAQI.Time = weatherAQI.Time.str.replace(
    'am', '').str.replace('pm', '').str.strip()

# COMBINE THE DATE & TIME COLUMNS INTO SINGLE TIMESTAMP:
weatherAQI.Date = pd.to_datetime(weatherAQI.Date.astype(
    str) + ' ' + weatherAQI.Time.astype(str))
weatherAQI = weatherAQI.drop(['Time'], axis=1)

time.sleep(2)
clear_output(wait=True)
print("Data Retrieval Complete!      ")
print("\U0001f31e"*13)
print('')
print("DataFrame = 'weatherAQI'")

weatherAQI.style.hide_index().set_properties(**{'text-align': 'left'})

sql_data = 'WeatherAQI.sqlite'
conn = sq.connect(sql_data)
cur = conn.cursor()
try:
    try:
        cur.executescript
        ('''
        CREATE TABLE weather (
            "index" INTEGER PRIMARY KEY AUTOINCREMENT,
            "City" CHAR NOT NULL,
            "Station" CHAR NOT NULL,
            "Date" DATE NOT NULL,
            "AQI" INTEGER NOT NULL,
            "Pollution_Range" CHAR NOT NULL,
            "Temp_f" FLOAT NOT NULL,
            "Pressure" INTEGER NOT NULL,
            "Humidity" INTEGER NOT NULL,
            "Wind_MPH" INTEGER NOT NULL,
            UNIQUE("Station","Date")
            );
        ''')
    
        weatherAQI.to_sql("weather", conn, if_exists='append', index=False)
    
    except:
        weatherAQI.to_sql("weather", conn, if_exists='append', index=False)
except:
    print('Duplicate data / records already exist in the database!')

conn.commit()
conn.close()


Data Retrieval Complete!      
🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞

DataFrame = 'weatherAQI'
